<a href="https://colab.research.google.com/github/crew8264/anyway-The-font-is-fixed./blob/main/Untotled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KoBERT finetuning

In [1]:
!pip install ipywidgets  # for vscode
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-u2ecya3u
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-u2ecya3u
     |████████████████████████████████| 132 kB 19.8 MB/s 
     |████████████████████████████████| 344 kB 50.8 MB/s 
     |████████████████████████████████| 49.1 MB 139 kB/s 
     |████████████████████████████████| 4.5 MB 47.4 MB/s 
     |████████████████████████████████| 1.2 MB 53.4 MB/s 
     |████████████████████████████████| 4.4 MB 5.3 MB/s 
     |████████████████████████████████| 101 kB 8.9 MB/s 
     |████████████████████████████████| 596 kB 11.1 MB/s 
     |████████████████████████████████| 6.6 MB 46.2 MB/s 
     |████████████████████████████████| 9.0 MB 39.1 MB/s 
     |███████

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

import pandas as pd

In [3]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [4]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
## CPU
# device = torch.device("cpu")

## GPU
device = torch.device("cuda:0")

In [6]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [7]:
!wget -O .cache/ratings_train.txt http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
!wget -O .cache/ratings_test.txt http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_test.txt

--2022-07-19 04:13:35--  http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
Resolving skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)... 52.219.146.75
Connecting to skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)|52.219.146.75|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘.cache/ratings_train.txt’

.cache/ratings_trai 100%[===================>]  13.95M  16.1MB/s    in 0.9s    

2022-07-19 04:13:36 (16.1 MB/s) - ‘.cache/ratings_train.txt’ saved [14628807/14628807]

--2022-07-19 04:13:36--  http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_test.txt
Resolving skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)... 52.219.146.75
Connecting to skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)|52.219.146.75|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (

In [8]:
# dataset_train = nlp.data.TSVDataset(".cache/ratings_train.txt", field_indices=[1,2], num_discard_samples=1)
# type(dataset_train)
# dataset_test = nlp.data.TSVDataset(".cache/ratings_test.txt", field_indices=[1,2], num_discard_samples=1)

In [81]:
####### 중요 :::: pandas XXX nlp로 읽어보기
origin_data = pd.read_csv('감성대화말뭉치_원시데이터__Training2.csv', encoding='cp949')
type(origin_data)

pandas.core.frame.DataFrame

In [143]:
data = origin_data

'''
data.loc[(data['감정_대분류'] == "기쁨"), '감정_대분류'] = 0
data.loc[(data['감정_대분류'] == "당황"), '감정_대분류'] = 1
data.loc[(data['감정_대분류'] == "분노"), '감정_대분류'] = 2
data.loc[(data['감정_대분류'] == "슬픔"), '감정_대분류'] = 3
data.loc[(data['감정_대분류'] == "불안"), '감정_대분류'] = 4
data.loc[(data['감정_대분류'] == "상처"), '감정_대분류'] = 5

'''
data.loc[(data['감정_소분류'] == "가난한, 불우한"), '감정_소분류'] = 0
data.loc[(data['감정_소분류'] == "감사하는"), '감정_소분류'] = 1
data.loc[(data['감정_소분류'] == "걱정스러운"), '감정_소분류'] = 2
data.loc[(data['감정_소분류'] == "고립된"), '감정_소분류'] = 3
data.loc[(data['감정_소분류'] == "괴로워하는"), '감정_소분류'] = 4
data.loc[(data['감정_소분류'] == "구역질 나는"), '감정_소분류'] = 5
data.loc[(data['감정_소분류'] == "기쁨"), '감정_소분류'] = 6
data.loc[(data['감정_소분류'] == "낙담한"), '감정_소분류'] = 7
data.loc[(data['감정_소분류'] == "남의 시선을 의식하는"), '감정_소분류'] = 8
data.loc[(data['감정_소분류'] == "노여워하는"), '감정_소분류'] = 9
data.loc[(data['감정_소분류'] == "눈물이 나는"), '감정_소분류'] = 10

data.loc[(data['감정_소분류'] == "느긋"), '감정_소분류'] = 11
data.loc[(data['감정_소분류'] == "당혹스러운"), '감정_소분류'] = 12
data.loc[(data['감정_소분류'] == "당황"), '감정_소분류'] = 13
data.loc[(data['감정_소분류'] == "두려운"), '감정_소분류'] = 14
data.loc[(data['감정_소분류'] == "마비된"), '감정_소분류'] = 15
data.loc[(data['감정_소분류'] == "만족스러운"), '감정_소분류'] = 16
data.loc[(data['감정_소분류'] == "방어적인"), '감정_소분류'] = 17
data.loc[(data['감정_소분류'] == "배신당한"), '감정_소분류'] = 18
data.loc[(data['감정_소분류'] == "버려진"), '감정_소분류'] = 19
data.loc[(data['감정_소분류'] == "부끄러운"), '감정_소분류'] = 20

data.loc[(data['감정_소분류'] == "분노"), '감정_소분류'] = 21
data.loc[(data['감정_소분류'] == "불안"), '감정_소분류'] = 22
data.loc[(data['감정_소분류'] == "비통한"), '감정_소분류'] = 23
data.loc[(data['감정_소분류'] == "상처"), '감정_소분류'] = 24
data.loc[(data['감정_소분류'] == "성가신"), '감정_소분류'] = 25
data.loc[(data['감정_소분류'] == "스트레스 받는"), '감정_소분류'] = 26
data.loc[(data['감정_소분류'] == "슬픔"), '감정_소분류'] = 27
data.loc[(data['감정_소분류'] == "신뢰하는"), '감정_소분류'] = 28
data.loc[(data['감정_소분류'] == "신이 난"), '감정_소분류'] = 29
data.loc[(data['감정_소분류'] == "실망한"), '감정_소분류'] = 30

data.loc[(data['감정_소분류'] == "악의적인"), '감정_소분류'] = 31
data.loc[(data['감정_소분류'] == "안달하는"), '감정_소분류'] = 32
data.loc[(data['감정_소분류'] == "안도"), '감정_소분류'] = 33
data.loc[(data['감정_소분류'] == "억울한"), '감정_소분류'] = 34
data.loc[(data['감정_소분류'] == "열등감"), '감정_소분류'] = 35
data.loc[(data['감정_소분류'] == "염세적인"), '감정_소분류'] = 36
data.loc[(data['감정_소분류'] == "외로운"), '감정_소분류'] = 37
data.loc[(data['감정_소분류'] == "우울한"), '감정_소분류'] = 38
data.loc[(data['감정_소분류'] == "자신하는"), '감정_소분류'] = 39
data.loc[(data['감정_소분류'] == "조심스러운"), '감정_소분류'] = 40

data.loc[(data['감정_소분류'] == "좌절한"), '감정_소분류'] = 41
data.loc[(data['감정_소분류'] == "죄책감의"), '감정_소분류'] = 42
data.loc[(data['감정_소분류'] == "질투하는"), '감정_소분류'] = 43
data.loc[(data['감정_소분류'] == "짜증내는"), '감정_소분류'] = 44
data.loc[(data['감정_소분류'] == "초조한"), '감정_소분류'] = 45
data.loc[(data['감정_소분류'] == "충격 받은"), '감정_소분류'] = 46
data.loc[(data['감정_소분류'] == "취약한"), '감정_소분류'] = 47
data.loc[(data['감정_소분류'] == "툴툴대는"), '감정_소분류'] = 48
data.loc[(data['감정_소분류'] == "편안한"), '감정_소분류'] = 49
data.loc[(data['감정_소분류'] == "한심한"), '감정_소분류'] = 50

data.loc[(data['감정_소분류'] == "혐오스러운"), '감정_소분류'] = 51
data.loc[(data['감정_소분류'] == "혼란스러운"), '감정_소분류'] = 52
data.loc[(data['감정_소분류'] == "환멸을 느끼는"), '감정_소분류'] = 53
data.loc[(data['감정_소분류'] == "회의적인"), '감정_소분류'] = 54
data.loc[(data['감정_소분류'] == "후회되는"), '감정_소분류'] = 55
data.loc[(data['감정_소분류'] == "흥분"), '감정_소분류'] = 56
data.loc[(data['감정_소분류'] == "희생된"), '감정_소분류'] = 57


data_list = []
#for ques, label in zip(data['사람문장1'], data['감정_대분류']) :
for ques, label in zip(data['사람문장1'], data['감정_소분류']) :
    data = []
    data.append(ques)
    data.append(str(label))

    data_list.append(data)

In [144]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [172]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        #print(dataset[label_idx])
        self.labels = [np.int32(i[label_idx]) for i in dataset]
        '''for i in dataset :
          print("asd : ", i)
          break'''

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [146]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [147]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
import random

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

index = random.randint(1 ,len(dataset_train))
print(dataset_train[index])

['아들놈이 또 사업에 실패했어.', '57']


In [148]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

['의사가 권해서 살을 찌웠는데 불필요한 살덩이같아.', '54']
asd :  ['요즘 부모님때문에 짜증나.', '44']
['취업했는데 다녀야 할지 고민이야.', '19']
asd :  ['은퇴하니 온종일 아무도 연락을 해오지 않아.', '54']


In [149]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [154]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=58,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [155]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [156]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [157]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [158]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [159]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [160]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().datSa.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [161]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/936 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 4.16969633102417 train acc 0.046875
epoch 1 batch id 201 loss 4.004156112670898 train acc 0.025575248756218905
epoch 1 batch id 401 loss 3.647982597351074 train acc 0.0755143391521197
epoch 1 batch id 601 loss 3.294618844985962 train acc 0.13259151414309484
epoch 1 batch id 801 loss 2.9404287338256836 train acc 0.177375936329588
epoch 1 train acc 0.19952681866744368


  0%|          | 0/234 [00:00<?, ?it/s]

epoch 1 test acc 0.3512731481481482


...

epoch 4 test acc 0.39414618945868946


  0%|          | 0/936 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 1.7498236894607544 train acc 0.59375
epoch 5 batch id 201 loss 1.907950758934021 train acc 0.5054415422885572
epoch 5 batch id 401 loss 2.382459878921509 train acc 0.5100140274314214
epoch 5 batch id 601 loss 1.9138394594192505 train acc 0.5159629783693843
epoch 5 batch id 801 loss 1.7195531129837036 train acc 0.5195458801498127
epoch 5 train acc 0.5202809343434344


  0%|          | 0/234 [00:00<?, ?it/s]

epoch 5 test acc 0.3950721153846154


In [184]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            '''
            if np.argmax(logits) == 0:
                test_eval.append("기쁨이")
            elif np.argmax(logits) == 1:
                test_eval.append("당황이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("불안이")
            elif np.argmax(logits) == 5:
                test_eval.append("상처가")
            '''

            if np.argmax(logits) == 0:
                test_eval.append("가난한, 불우한")
            elif np.argmax(logits) == 1:
                test_eval.append("감사하는")
            elif np.argmax(logits) == 2:
                test_eval.append("걱정스러운")
            elif np.argmax(logits) == 3:
                test_eval.append("고립된")
            elif np.argmax(logits) == 4:
                test_eval.append("괴로워하는")
            elif np.argmax(logits) == 5:
                test_eval.append("구역질 나는")
            elif np.argmax(logits) == 6:
                test_eval.append("기쁨의")
            elif np.argmax(logits) == 7:
                test_eval.append("낙담한")
            elif np.argmax(logits) == 8:
                test_eval.append("남의 시선을 의식하는")
            elif np.argmax(logits) == 9:
                test_eval.append("노여워하는")
            elif np.argmax(logits) == 10: ## 10 ##
                test_eval.append("눈물이 나는")
            elif np.argmax(logits) == 11:
                test_eval.append("느긋한")
            elif np.argmax(logits) == 12:
                test_eval.append("당혹스러운")
            elif np.argmax(logits) == 13:
                test_eval.append("당황한")
            elif np.argmax(logits) == 14:
                test_eval.append("두려운")
            elif np.argmax(logits) == 15:
                test_eval.append("마비된")
            elif np.argmax(logits) == 16:
                test_eval.append("만족스러운")
            elif np.argmax(logits) == 17:
                test_eval.append("방어적인")
            elif np.argmax(logits) == 18:
                test_eval.append("배신당한")
            elif np.argmax(logits) == 19:
                test_eval.append("버려진")
            elif np.argmax(logits) == 20: ## 20 ##
                test_eval.append("부끄러운")
            elif np.argmax(logits) == 21:
                test_eval.append("분노한")
            elif np.argmax(logits) == 22:
                test_eval.append("불안한")
            elif np.argmax(logits) == 23:
                test_eval.append("비통한")
            elif np.argmax(logits) == 24:
                test_eval.append("상처 받은")
            elif np.argmax(logits) == 25:
                test_eval.append("성가신")
            elif np.argmax(logits) == 26:
                test_eval.append("스트레스 받는")
            elif np.argmax(logits) == 27:
                test_eval.append("슬픔의")
            elif np.argmax(logits) == 28:
                test_eval.append("신뢰하는")
            elif np.argmax(logits) == 29:
                test_eval.append("신이 난")
            elif np.argmax(logits) == 30: ## 30 ##
                test_eval.append("실망한")
            elif np.argmax(logits) == 31:
                test_eval.append("악의적인")
            elif np.argmax(logits) == 32:
                test_eval.append("안달하는")
            elif np.argmax(logits) == 33:
                test_eval.append("안도")
            elif np.argmax(logits) == 34:
                test_eval.append("억울한")
            elif np.argmax(logits) == 35:
                test_eval.append("열등감")
            elif np.argmax(logits) == 36:
                test_eval.append("염세적인")
            elif np.argmax(logits) == 37:
                test_eval.append("외로운")
            elif np.argmax(logits) == 38:
                test_eval.append("우울한")
            elif np.argmax(logits) == 39:
                test_eval.append("자신하는")
            elif np.argmax(logits) == 40: ## 40 ##
                test_eval.append("조심스러운")
            elif np.argmax(logits) == 41:
                test_eval.append("좌절한")
            elif np.argmax(logits) == 42:
                test_eval.append("죄책감의")
            elif np.argmax(logits) == 43:
                test_eval.append("질투하는")
            elif np.argmax(logits) == 44:
                test_eval.append("짜증내는")
            elif np.argmax(logits) == 45:
                test_eval.append("초조한")
            elif np.argmax(logits) == 46:
                test_eval.append("충격 받은")
            elif np.argmax(logits) == 47:
                test_eval.append("취약한")
            elif np.argmax(logits) == 48:
                test_eval.append("툴툴대는")
            elif np.argmax(logits) == 49:
                test_eval.append("편안한")
            elif np.argmax(logits) == 50: # 50
                test_eval.append("한심한")
            elif np.argmax(logits) == 51:
                test_eval.append("혐오스러운")
            elif np.argmax(logits) == 52:
                test_eval.append("혼란스러운")
            elif np.argmax(logits) == 53:
                test_eval.append("환멸을 느끼는")
            elif np.argmax(logits) == 54:
                test_eval.append("회의적인")
            elif np.argmax(logits) == 55:
                test_eval.append("후회되는")
            elif np.argmax(logits) == 56:
                test_eval.append("흥분한")
            elif np.argmax(logits) == 57:
                test_eval.append("희생된")



            if np.argmax(logits) == 0:
              test_eval.append()


        # print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")
        print("> \"" + test_eval[0] + "\" 기운이 느껴집니다.")

In [185]:
#질문 무한반복, 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")
    

하고싶은 말을 입력해주세요 : 안녕?


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


> "편안한" 기운이 느껴집니다.


하고싶은 말을 입력해주세요 : 0
